## Questions to Answer
1. Does Filtering on noise estimates worsen imbalance? Before vs after IR for CLNI, Two-stage ensemble
2. Avg. noise likelihood of minority vs majority class?
3. Ratio of 0->1 & 1->0 noise types vs ratio of noisy samples identified in each class. With oracle access to no of actual, total no. of noisy samples 

In [25]:
import numpy as np,os
import pandas as pd
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score, precision_score, recall_score
import seaborn as sns,matplotlib.pyplot as plt

from library.utils import evaluate, read_data
from library.cleaners import kDN, ih_prob

## Notes
+ Using `kDN(K=5)`: In `25/32` datasets, no of samples with NP noise (Bug class in noisy dataset but actually Clean) has been overestimated. Consequently PN has been underestimated. This shows minority class gets disproportionately affected.
    + For `ih_prob`: that number is `23/32`
+ In terms of avg. probability of being noisy: samples in Bug class (of noisy data version) have vastly hugher value than clean ones. On further avg on 32 datasets, `.702 vs .074`, almost 10 times higher.

In [26]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [27]:
DATASETS.index("camel-1.4.0.csv")

25

In [ ]:
df = pd.DataFrame(index=DATASETS,columns=['PN','ePN','NP','eNP','avg-bug','avg-clean'])
for d in DATASETS:
    X,y_noisy,y_real = read_data(d,True)
    PN = y_noisy[y_real==1]==0   #So, really 1(P), but now 0(N)
    NP = y_noisy[y_real==0]==1   #So, really 0(N), but now 1(P)
    assert PN.sum()+NP.sum()==(y_noisy!=y_real).sum()
    print(f"PN {PN.sum()},NP {NP.sum()}")
    ne = kDN(X,y_noisy,K=5)
    avg_bug,avg_clean = ne[y_noisy==1].mean(),ne[y_noisy==0].mean()  #avg_bug: avg noise probs of those labelled Buggy in Noisy dataset
    actual_total = (y_noisy!=y_real).sum()
    nidx = np.argsort(ne)[-actual_total:]
    eNP = y_noisy[nidx]==1
    ePN = y_noisy[nidx]==0
    print(f"ePN {ePN.sum()},eNP {eNP.sum()}")
    print(f"Avg, Bug:{avg_bug:.3f}, Clean:{avg_clean:.3f}")
    print()
    df.loc[d,:] = [x.sum() for x in [PN,ePN,NP,eNP]] + [avg_bug,avg_clean]

In [ ]:
df

In [ ]:
plt.figure(figsize=(18,5))
plt.plot(range(len(DATASETS)),df['PN'],c='r',label='Actual');
plt.plot(range(len(DATASETS)),df['ePN'],c='b',label='Predicted');
plt.legend();

In [ ]:
(df['ePN']>df['PN']).sum(),(df['eNP']>df['NP']).sum()

In [ ]:
plt.figure(figsize=(18,5))
plt.plot(range(len(DATASETS)),df['NP'],c='r',label='Actual');
plt.plot(range(len(DATASETS)),df['eNP'],c='b',label='Predicted');
plt.legend();

In [ ]:
plt.figure(figsize=(18,5))
plt.scatter(range(len(DATASETS)),df['avg-bug'],c='r',label='Buggy');
plt.scatter(range(len(DATASETS)),df['avg-clean'],c='b',label='Clean');
plt.legend();

In [ ]:
df['avg-bug'].mean(),df['avg-clean'].mean()

## Impact on Classification performance
So, both noise detectors discriminate against minority Bug class. But when we incorporate this noise estimates into this imbalanced classification setting, how much does this actually hurt classification performance? Is balancing first better? `Balancing->Filtering` vs `Filtering->Balancing`

In [4]:
from collections import defaultdict
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import BalancedBaggingClassifier, RUSBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
from imblearn.pipeline import Pipeline

from library.cleaners import FilteringEstimator

In [13]:
def evaluateFilter(clf,X,y_noisy,y_real,cv,scorers):
    scores = defaultdict(list)
    to_keep = (y_noisy==y_real).sum()
    for train_id, test_id in cv.split(X,y_real):  #vs y_noisy, to solve no-pos-label-in-test-set bug     
        try:
            clf = clf.fit(X[train_id],y_noisy[train_id],to_keep=to_keep)
            probs = clf.predict_proba(X[test_id])
            assert probs.shape[1]==2
        except (ValueError,AssertionError) as e: 
            #assert str(e).startswith("Expected n_neighbors") or str(e).startswith("The target"),f"Some different Error:{str(e)}"
            for func in scorers:
                scores[func.__name__].append(0)
            print("ERROR")
            continue
        labels = np.argmax(probs,axis=1)
        for func in scorers:
            yp = probs[:,1]
            try:
                func([0,1,1],[.2,.6,.7])
                yp = probs[:,1]
            except ValueError as e:
                yp = labels
            scores[func.__name__].append(func(y_real[test_id],yp))
    for func in scorers:
        scores[func.__name__] = np.array(scores[func.__name__])
    return scores

In [6]:
imbs = {
    'smote': SMOTE(k_neighbors=5),
    'rus': RandomUnderSampler('not minority'),
    'wilson':EditedNearestNeighbours(n_neighbors=5),  #Default was 3
    'tomek': TomekLinks(),
}
clfs = {
    'dt': DecisionTreeClassifier(max_depth=20),
    'lr': LogisticRegression(solver='lbfgs',max_iter=1000),
    'nb': GaussianNB(),
    'knn': KNeighborsClassifier(n_neighbors=5),
    'rf': RandomForestClassifier(n_estimators=50),
}
ensembles = {
    'rboost_DT': RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10),algorithm='SAMME',n_estimators=10),
    'rboost_NB': RUSBoostClassifier(base_estimator=GaussianNB(),algorithm='SAMME',n_estimators=10),
    'bbag_DT': BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=20,max_features='sqrt')),
    'bbag_NB': BalancedBaggingClassifier(base_estimator=GaussianNB()),
}
simples = {
    'LR': LogisticRegression(solver='lbfgs',max_iter=1000),
    'RF': RandomForestClassifier(n_estimators=50)
}

## Filtering -> Balancing

In [7]:
models = {}
for im,samp in imbs.items():
    for c,clf in clfs.items():
        models[(im,c)] = FilteringEstimator(Pipeline([('samp',samp),('clf',clf)]),kDN,K=5)

for m,ens in ensembles.items():
    models[('ens',m)] = FilteringEstimator(ens,kDN,K=5)
    
for m,clf in simples.items():
    models[('sim',m)] = FilteringEstimator(clf,kDN,K=5)    
    
models.keys(),len(models)

(dict_keys([('smote', 'dt'), ('smote', 'lr'), ('smote', 'nb'), ('smote', 'knn'), ('smote', 'rf'), ('rus', 'dt'), ('rus', 'lr'), ('rus', 'nb'), ('rus', 'knn'), ('rus', 'rf'), ('wilson', 'dt'), ('wilson', 'lr'), ('wilson', 'nb'), ('wilson', 'knn'), ('wilson', 'rf'), ('tomek', 'dt'), ('tomek', 'lr'), ('tomek', 'nb'), ('tomek', 'knn'), ('tomek', 'rf'), ('ens', 'rboost_DT'), ('ens', 'rboost_NB'), ('ens', 'bbag_DT'), ('ens', 'bbag_NB'), ('sim', 'LR'), ('sim', 'RF')]),
 26)

In [8]:
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=None)
def pr_rec_score(y,yp):
    prec, rec, _ = precision_recall_curve(y,yp)
    return auc(rec,prec)
scorers = [matthews_corrcoef,pr_rec_score]

In [ ]:
cols = pd.MultiIndex.from_product([imbs.keys(),clfs.keys(),[f.__name__ for f in scorers]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)
df = pd.read_csv("Filtering->Balancing.csv",header=[0,1,2],index_col=0)

In [14]:
for d in DATASETS:
    X,y_noisy,y_real = read_data(d,stats=True)
    if df.loc[d,:].isna().sum()==0: 
        print(f"Skipping {d}")
        continue
    for k in models:
        print(k)
        r = evaluateFilter(models[k],X,y_noisy,y_real,cv,scorers)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()
    df.to_csv("Filtering->Balancing.csv")

activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)
Skipping activemq-5.8.0.csv
groovy-1_6_BETA_1.csv noise:0.128, imb:6.017,117,704, Shape:(821, 65)
Skipping groovy-1_6_BETA_1.csv
activemq-5.3.0.csv noise:0.094, imb:15.669,142,2225, Shape:(2367, 65)
Skipping activemq-5.3.0.csv
wicket-1.3.0-incubating-beta-1.csv noise:0.164, imb:4.806,288,1384, Shape:(1672, 65)
Skipping wicket-1.3.0-incubating-beta-1.csv
jruby-1.1.csv noise:0.175, imb:3.540,161,570, Shape:(731, 65)
Skipping jruby-1.1.csv
jruby-1.4.0.csv noise:0.190, imb:3.890,200,778, Shape:(978, 65)
Skipping jruby-1.4.0.csv
lucene-2.3.0.csv noise:0.204, imb:4.031,160,645, Shape:(805, 65)
Skipping lucene-2.3.0.csv
hbase-0.95.2.csv noise:0.260, imb:15.088,114,1720, Shape:(1834, 65)
('smote', 'dt')
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
('smote', 'lr')
ERROR
ERROR
ERROR
ERROR
ERROR

/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


('rus', 'knn')
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
('rus', 'rf')
('wilson', 'dt')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = 

('wilson', 'lr')


...

/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


ERROR


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


ERROR


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = 

In [15]:
df

imb                                            smote               \
clf                                               dt                
metric                             matthews_corrcoef pr_rec_score   
activemq-5.8.0.csv                          0.162811     0.240504   
groovy-1_6_BETA_1.csv                       0.268819     0.357761   
activemq-5.3.0.csv                          0.274237     0.377514   
wicket-1.3.0-incubating-beta-1.csv          0.226227     0.338274   
jruby-1.1.csv                               0.413860     0.541788   
jruby-1.4.0.csv                             0.387128     0.544849   
lucene-2.3.0.csv                            0.514840     0.687570   
hbase-0.95.2.csv                            0.000000     0.000000   
lucene-3.0.0.csv                            0.178082     0.313141   
camel-2.9.0.csv                             0.131233     0.135190   
wicket-1.5.3.csv                            0.094826     0.141113   
lucene-3.1.csv                              0.159299     0.226974   
groovy-1_6_BETA_2.csv                       0.233343     0.324400   
activemq-5.2.0.csv                          0.243076     0.357179   
groovy-1_5_7.csv                            0.148682     0.262351   
hbase-0.95.0.csv                            0.000000     0.020459   
camel-2.11.0.csv                            0.200229     0.152263   
jruby-1.7.0.preview1.csv                    0.187552     0.295543   
hive-0.10.0.csv                             0.087594     0.360778   
camel-2.10.0.csv                            0.128852     0.161197   
derby-10.2.1.6.csv                          0.052377     0.224013   
jruby-1.5.0.csv                             0.324521     0.473782   
derby-10.3.1.4.csv                          0.000000     0.000000   
lucene-2.9.0.csv                            0.264952     0.465526   
hbase-0.94.0.csv                            0.124971     0.325005   
camel-1.4.0.csv                             0.273182     0.459033   
activemq-5.1.0.csv                          0.245327     0.323816   
activemq-5.0.0.csv                          0.199529     0.566297   
derby-10.5.1.1.csv                          0.242955     0.386368   
wicket-1.3.0-beta2.csv                      0.209824     0.323928   
hive-0.12.0.csv                             0.054434     0.149199   
hive-0.9.0.csv                              0.000000     0.000000   

imb                                                                \
clf                                               lr                
metric                             matthews_corrcoef pr_rec_score   
activemq-5.8.0.csv                          0.255446     0.304742   
groovy-1_6_BETA_1.csv                       0.400412     0.428778   
activemq-5.3.0.csv                          0.375578     0.448824   
wicket-1.3.0-incubating-beta-1.csv          0.311682     0.415102   
jruby-1.1.csv                               0.413151     0.574261   
jruby-1.4.0.csv                             0.413216     0.569409   
lucene-2.3.0.csv                            0.584185     0.741042   
hbase-0.95.2.csv                            0.012447     0.042285   
lucene-3.0.0.csv                            0.266700     0.344647   
camel-2.9.0.csv                             0.187580     0.200366   
wicket-1.5.3.csv                            0.166947     0.159984   
lucene-3.1.csv                              0.173419     0.233503   
groovy-1_6_BETA_2.csv                       0.294991     0.358358   
activemq-5.2.0.csv                          0.355251     0.432789   
groovy-1_5_7.csv                            0.210065     0.400640   
hbase-0.95.0.csv                            0.000000     0.000000   
camel-2.11.0.csv                            0.167163     0.142760   
jruby-1.7.0.preview1.csv                    0.243285     0.292813   
hive-0.10.0.csv                             0.137336     0.317670   
camel-2.10.0.csv                            0.186298     0.188838 

In [19]:
from scipy.stats import wilcoxon, friedmanchisquare, rankdata
import scikit_posthocs as sp

In [20]:
mat = df.drop(columns=['matthews_corrcoef'],axis=1,level=2).droplevel(2,axis=1)
mat.shape

(32, 26)

In [21]:
res = mat.copy()
for d in mat.index:
    res.loc[d] = rankdata(1-mat.loc[d])
res

imb                                smote                           rus        \
clf                                   dt    lr    nb   knn    rf    dt    lr   
activemq-5.8.0.csv                  26.0  14.0   3.0  20.0  19.0   1.0  15.0   
groovy-1_6_BETA_1.csv               25.0  15.0  17.0  11.0  10.0   6.0  23.0   
activemq-5.3.0.csv                  25.0  15.0  12.0  24.0   4.0   8.0  17.0   
wicket-1.3.0-incubating-beta-1.csv  21.0   9.0   4.0  14.0  26.0   6.0  12.0   
jruby-1.1.csv                       26.0  24.0   5.0  18.0  16.0  25.0  17.0   
jruby-1.4.0.csv                     22.0  17.0   2.0  19.0   6.0  24.0  18.0   
lucene-2.3.0.csv                    19.0  10.0  22.0  13.0   1.0  20.0  12.0   
hbase-0.95.2.csv                    25.5  23.0  24.0  25.5  22.0  15.0  16.0   
lucene-3.0.0.csv                    15.0   3.0  10.0   6.0  20.0   1.0  14.0   
camel-2.9.0.csv                     25.0  19.0   3.0  14.0  23.0   5.0  24.0   
wicket-1.5.3.csv                    25.0  19.0   3.0  15.0  23.0   1.0  24.0   
lucene-3.1.csv                      14.0  11.0   4.0   7.0  24.0   1.0  15.0   
groovy-1_6_BETA_2.csv               24.0  22.0  18.0  11.0   5.0   4.0  26.0   
activemq-5.2.0.csv                  25.0  15.0   3.0  24.0  10.0  13.0  11.0   
groovy-1_5_7.csv                    26.0  17.0   4.0  22.0  16.0  20.0  15.0   
hbase-0.95.0.csv                    22.0  24.5  24.5  24.5  24.5  13.0  16.0   
camel-2.11.0.csv                    24.0  25.0   8.0  13.0   2.0   1.0  26.0   
jruby-1.7.0.preview1.csv            22.0  23.0   3.0  21.0  26.0   4.0  20.0   
hive-0.10.0.csv                     18.0  25.0   5.0  24.0  23.0  13.0   8.0   
camel-2.10.0.csv                    25.0  19.0   4.0  15.0  23.0   2.0  22.0   
derby-10.2.1.6.csv                  23.0  25.0  24.0  26.0  22.0  15.0   1.0   
jruby-1.5.0.csv                     17.0  22.0   5.0  14.0  25.0   9.0  23.0   
derby-10.3.1.4.csv                  23.0  23.0  23.0  23.0  23.0   4.0  15.0   
lucene-2.9.0.csv                    13.0  10.0   6.0   7.0  22.0   1.0   8.0   
hbase-0.94.0.csv                    23.0  21.0  24.0  25.0  26.0  15.0  11.0   
camel-1.4.0.csv                     23.0  20.0   1.0  25.0  16.0   2.0  18.0   
activemq-5.1.0.csv                  23.0  21.0   3.0  25.0  14.0   6.0  26.0   
activemq-5.0.0.csv                   1.0  26.0  18.0  11.0   7.0  16.0  10.0   
derby-10.5.1.1.csv                  26.0  17.0   4.0  25.0   9.0  14.0  20.0   
wicket-1.3.0-beta2.csv              17.0  19.0   3.0  10.0  25.0   5.0  18.0   
hive-0.12.0.csv                     25.0  13.0   3.0  15.0  11.0   1.0  24.0   
hive-0.9.0.csv                      24.0  24.0  24.0  24.0  24.0  17.0   1.0   

imb                                                   ... tomek              \
clf                                   nb   knn    rf  ...    lr    nb   knn   
activemq-5.8.0.csv                   5.0  21.0  17.0  ...   8.0   4.0  23.0   
groovy-1_6_BETA_1.csv               21.0   7.0  12.0  ...  13.0  20.0  22.0   
activemq-5.3.0.csv                  14.0  22.0  20.0  ...   5.0  13.0  23.0   
wicket-1.3.0-incubating-beta-1.csv   3.0  18.0  24.0  ...   8.0   2.0  22.0   
jruby-1.1.csv                       10.0   7.0  13.0  ...  14.0   8.0   3.0   
jruby-1.4.0.csv                      5.0  23.0  11.0  ...  13.0   4.0  15.0   
lucene-2.3.0.csv                    26.0  14.0   5.0  ...   8.0  25.0  16.0   
hbase-0.95.2.csv                     5.0  18.0  13.0  ...  11.0   9.0   1.0   
lucene-3.0.0.csv                     7.0  22.0  18.0  ...  11.0  12.0  26.0   
camel-2.9.0.csv                      4.0   8.0  21.0  ...  10.0   6.0  13.0   
wicket-1.5.3.csv                     6.0  14.0  22.0  ...   8.0   5.0  26.0   
lucene-3.1.csv                       5.0  17.0  25.0  ...  13.0   6.0  26.0   
groovy-1_6_BETA_2.csv               20.0   9.0  10.0  ...  15.0  21.0   6.0   
activemq-5.2.0.csv                   5.0  23.0  19.0  ...   7.0   4.0  22.0   
groovy-1_5_7.cs

In [24]:
res.mean().sort_values()

imb     clf      
wilson  nb            7.656250
tomek   nb            8.468750
rus     nb            8.843750
        dt            9.000000
wilson  lr            9.687500
smote   nb            9.890625
wilson  rf           10.062500
sim     LR           10.281250
wilson  dt           10.843750
tomek   lr           11.312500
sim     RF           12.437500
ens     bbag_NB      12.562500
tomek   rf           12.687500
wilson  knn          12.843750
rus     rf           14.906250
        knn          15.156250
ens     rboost_DT    15.687500
        rboost_NB    16.375000
rus     lr           16.437500
tomek   dt           16.437500
        knn          16.593750
smote   rf           17.109375
ens     bbag_DT      17.781250
smote   knn          17.843750
        lr           18.453125
        dt           21.640625
dtype: float64